This is the initial notebook to explore the data and run the first models. 

In [1]:
import pandas as pd
import numpy as np

from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestRegressor
import xgboost as xgb
from sklearn.preprocessing import OneHotEncoder

from fastai.imports import *
from fastai.tabular.all import *

from numpy import random
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_squared_log_error
from sklearn import metrics
from sklearn.model_selection import KFold, cross_val_score

import eli5
from eli5.sklearn import PermutationImportance

from scipy.stats import skew, norm
from scipy.special import boxcox1p
from scipy.stats import boxcox_normmax

In [2]:
test = pd.read_csv('test.csv')
sample = pd.read_csv('sample_submission.csv')
train = pd.read_csv('train.csv')

Based on initial analysis the train and test datasets have similar characteristics, so it will be easier to combine them for imputation and data analysis work. 

In [3]:
train_test = pd.concat([train, test], ignore_index=True)

In [4]:
pd.isnull(train_test).sum()[pd.isnull(train_test).sum() > 0]

MSZoning           4
LotFrontage      486
Alley           2721
Utilities          2
Exterior1st        1
Exterior2nd        1
MasVnrType        24
MasVnrArea        23
BsmtQual          81
BsmtCond          82
BsmtExposure      82
BsmtFinType1      79
BsmtFinSF1         1
BsmtFinType2      80
BsmtFinSF2         1
BsmtUnfSF          1
TotalBsmtSF        1
Electrical         1
BsmtFullBath       2
BsmtHalfBath       2
KitchenQual        1
Functional         2
FireplaceQu     1420
GarageType       157
GarageYrBlt      159
GarageFinish     159
GarageCars         1
GarageArea         1
GarageQual       159
GarageCond       159
PoolQC          2909
Fence           2348
MiscFeature     2814
SaleType           1
SalePrice       1459
dtype: int64

It looks like Alley, FireplaceQu, PoolQC, Fence and MiscFeature have significant numbers of missing data. So it will be best to eliminate those rows. There are a number of rows that have less than 5 rows with missing data. Since some of these are categorical and some are continuous data, their missing data will be replaced with the most frequent value. 

In [5]:
drop_high_nan=['Alley','FireplaceQu','PoolQC','Fence','MiscFeature']
train_test=train_test.drop(drop_high_nan,axis=1)
small_nan_cols = ['MSZoning', 'Utilities', 'Exterior1st', 'Exterior2nd', 'BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF', 
                  'TotalBsmtSF', 'Electrical', 'BsmtFullBath', 'BsmtHalfBath', 'KitchenQual', 'Functional', 'GarageCars', 
                  'GarageArea','SaleType', 'SaleCondition']
small_impute = SimpleImputer(strategy='most_frequent')
train_test[small_nan_cols] = pd.DataFrame(small_impute.fit_transform(train_test[small_nan_cols]),columns=small_nan_cols)

The following columns seem to have one value significantly larger than the rest, and it would probably be best to use the mode, or most common, value to feel each NaN value: MasVnrType, MasVnrArea, BsmtCond, BsmtExposure, BsmtFinType2, GarageType, GarageFinish, GarageQual, and GarageCond. That is represents 9 out of the 13 columns. 

BsmtQual has two values larger than the rest: Gd and TA. But it only has 2.8% NaNs, so simply using the mode might be good enough.  

GarageYrBlt has 59 NaNs out 2919 rows which is only 2%. It has a dispersed set of values, so it might be easiest just to have any NaNs have the same value as YearBuilt. 

BsmtFinType1 has only 2.7% value of NaNs, and most two of its largest values are GLQ and Unf. It might be easiest to use the mode here. 

LotFrontage has 486 NaNs out of 2919 rows which is a pretty high 16.7%. It has a dispersed range of values, but looking at its characteristics from the describe function above, it seems to have a pretty even distribution with a mean of 10,168 and a median of 9,453. So using the mean to fill in the NaNs seems reasonable. If it turns out this value has a high impact on the predictability of the SalePrice, then it might be good to revisit this assumption. 

In [6]:
mode_cols = ['MasVnrType', 'MasVnrArea', 'BsmtCond', 'BsmtExposure', 'BsmtFinType2', 'GarageType', 'GarageFinish', 
             'GarageQual','GarageCond', 'BsmtQual', 'BsmtFinType1']
mode_impute = SimpleImputer(strategy='most_frequent')
train_test[mode_cols] = pd.DataFrame(mode_impute.fit_transform(train_test[mode_cols]),columns=mode_cols)
train_test['LotFrontage'].fillna((train_test['LotFrontage'].mean()), inplace=True)
train_test['GarageYrBlt'] = train_test['GarageYrBlt'].fillna(train_test['YearBuilt'])

In [7]:
pd.isnull(train_test).sum()[pd.isnull(train_test).sum() > 0]

SalePrice    1459
dtype: int64

In [8]:
train_test['BsmtQual'].unique()

array(['Gd', 'TA', 'Ex', 'Fa'], dtype=object)

In [9]:
train_test['BsmtFinType1'].unique()

array(['GLQ', 'ALQ', 'Unf', 'Rec', 'BLQ', 'LwQ'], dtype=object)

In [10]:
train_test.BsmtQual = train_test.BsmtQual.replace({"Ex": 110, "Gd": 95, "TA": 85, "Fa": 75, "Po": 60, "NA": 0})

In [11]:
train_test.BsmtFinType1 = train_test.BsmtFinType1.replace({"GLQ": 6, "ALQ": 5, "BLQ": 4, "Rec": 3, "LwQ": 2, "Unf": 1,
                                                         "NA": 0})

It will be necessary to identify all the columns that have non-numeric object values and then convert them to numeric values. 

In [12]:
obj_cols = list(train_test.select_dtypes(['object']).columns)
obj_cols

['MSZoning',
 'Street',
 'LotShape',
 'LandContour',
 'Utilities',
 'LotConfig',
 'LandSlope',
 'Neighborhood',
 'Condition1',
 'Condition2',
 'BldgType',
 'HouseStyle',
 'RoofStyle',
 'RoofMatl',
 'Exterior1st',
 'Exterior2nd',
 'MasVnrType',
 'MasVnrArea',
 'ExterQual',
 'ExterCond',
 'Foundation',
 'BsmtCond',
 'BsmtExposure',
 'BsmtFinSF1',
 'BsmtFinType2',
 'BsmtFinSF2',
 'BsmtUnfSF',
 'TotalBsmtSF',
 'Heating',
 'HeatingQC',
 'CentralAir',
 'Electrical',
 'BsmtFullBath',
 'BsmtHalfBath',
 'KitchenQual',
 'Functional',
 'GarageType',
 'GarageFinish',
 'GarageCars',
 'GarageArea',
 'GarageQual',
 'GarageCond',
 'PavedDrive',
 'SaleType',
 'SaleCondition']

In [13]:
for column in obj_cols:
     train_test[column] = pd.factorize(train_test[column], sort=True)[0]

C:\Users\tompe\anaconda3\lib\site-packages\pandas\core\algorithms.py:798: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)
  uniques = Index(uniques)


To create some new columns that might compound the effects of the existing columns with higher impact on the xgb score. 

In [14]:
train_test['QualCondSum'] = train_test['OverallQual'] + train_test['OverallCond']
train_test['RemodTime'] = train_test['YearRemodAdd'] - train_test['YearBuilt']
train_test['BsmtFinTypeSF1'] = train_test['BsmtFinType1'] * train_test['BsmtFinSF1']
train_test['TotalFlrSF'] = train_test['1stFlrSF'] + train_test['2ndFlrSF']
train_test['TotalFinSF'] = train_test['GrLivArea'] + train_test['BsmtFinSF1']
train_test['GarageCarArea'] = train_test['GarageArea'] * train_test['GarageCars']
train_test['TotalSF'] = train_test['1stFlrSF'] + train_test['2ndFlrSF'] + train_test['TotalBsmtSF']

To now eliminate the columns that have negative or zero influence on the xgb score. 

In [15]:
train_test.drop(['Utilities','Street','TotRmsAbvGrd','3SsnPorch','YrSold','Exterior2nd','LotConfig',
                              'HouseStyle','EnclosedPorch','WoodDeckSF','Foundation','RoofMatl','Electrical',
                              'GarageType','LotFrontage','SaleType','MoSold','BsmtExposure','1stFlrSF',
                              'BsmtFinSF1','Exterior1st','KitchenQual','TotalFlrSF'], axis=1, inplace=True)

To look at how many features have a skew above 0.5, since high skew can be an issue in regression analysis. 

In [16]:
skew_features = train_test.apply(lambda x: skew(x)).sort_values(ascending=False)

In [17]:
high_skew = skew_features[skew_features > 0.5]
skew_index = high_skew.index

In [18]:
skew_index

Index(['MiscVal', 'PoolArea', 'LotArea', 'LowQualFinSF', 'Heating',
       'Condition2', 'LandSlope', 'KitchenAbvGr', 'ScreenPorch',
       'BsmtHalfBath', 'BsmtFinSF2', 'Condition1', 'OpenPorchSF', 'BldgType',
       'RemodTime', 'MasVnrArea', 'RoofStyle', 'MSSubClass', 'GarageCarArea',
       'GrLivArea', 'TotalFinSF', 'BsmtFinTypeSF1', 'TotalSF', '2ndFlrSF',
       'BsmtQual', 'Fireplaces', 'HalfBath', 'BsmtFullBath', 'OverallCond'],
      dtype='object')

In [19]:
print("There are {} numerical features with Skew > 0.5 :".format(high_skew.shape[0]))
skewness = pd.DataFrame({'Skew' :high_skew})
skew_features.head(30)

There are 29 numerical features with Skew > 0.5 :


MiscVal           21.947195
PoolArea          16.898328
LotArea           12.822431
LowQualFinSF      12.088761
Heating           12.078788
Condition2        12.060093
LandSlope          4.975157
KitchenAbvGr       4.302254
ScreenPorch        3.946694
BsmtHalfBath       3.931594
BsmtFinSF2         3.476562
Condition1         2.983114
OpenPorchSF        2.535114
BldgType           2.192261
RemodTime          2.063712
MasVnrArea         1.556310
RoofStyle          1.553307
MSSubClass         1.375457
GarageCarArea      1.296672
GrLivArea          1.269358
TotalFinSF         1.159630
BsmtFinTypeSF1     0.932490
TotalSF            0.922771
2ndFlrSF           0.861675
BsmtQual           0.796302
Fireplaces         0.733495
HalfBath           0.694566
BsmtFullBath       0.624832
OverallCond        0.570312
HeatingQC          0.486656
dtype: float64

To normalize the features with skew above 0.5.

In [20]:
for i in skew_index:
    train_test[i] = np.log1p(train_test[i])

C:\Users\tompe\anaconda3\lib\site-packages\pandas\core\arraylike.py:397: RuntimeWarning: divide by zero encountered in log1p
  result = getattr(ufunc, method)(*inputs, **kwargs)


To create a column with the log of the SalePrice to match the evaluation process in the contest. 

In [22]:
train_test['LogSalePrice'] = train_test['SalePrice'].apply(np.log)

In [23]:
train_test['LogSalePrice'].head()

0    12.247694
1    12.109011
2    12.317167
3    11.849398
4    12.429216
Name: LogSalePrice, dtype: float64

In [24]:
train_test.head()

,Id,MSSubClass,MSZoning,LotArea,LotShape,LandContour,LandSlope,Neighborhood,Condition1,Condition2,...,MiscVal,SaleCondition,SalePrice,QualCondSum,RemodTime,BsmtFinTypeSF1,TotalFinSF,GarageCarArea,TotalSF,LogSalePrice
0,1,4.110874,3,9.042040,3,3,0.0,5,1.098612,1.098612,...,0.0,4,208500.0,12,0.000000,8.030410,7.706613,6.340359,7.586804,12.247694
1,2,3.044522,3,9.169623,3,3,0.0,24,0.693147,1.098612,...,0.0,4,181500.0,14,0.000000,8.183397,7.590347,6.003887,7.531552,12.109011
2,3,4.110874,3,9.328212,0,3,0.0,5,1.098612,1.098612,...,0.0,4,223500.0,12,0.693147,7.597396,7.658700,6.514713,7.646831,12.317167
3,4,4.262680,3,9.164401,0,3,0.0,6,1.098612,1.098612,...,0.0,0,140000.0,12,4.025352,6.364751,7.514255,7.002156,7.549083,11.849398
4,5,4.110874,3,9.565284,0,3,0.0,15,1.098612,1.098612,...,0.0,4,250000.0,13,0.000000,7.942718,7.889084,7.303843,7.902118,12.429216


To separate the train_test dataset back into the train and test datasets, identify the independent and dependent columns, and create the validation split.

In [25]:
train = train_test[train_test['SalePrice'].notnull()].copy()
test = train_test[train_test['SalePrice'].isnull()].drop(['SalePrice','LogSalePrice'],axis=1)
X = train.drop(['SalePrice','LogSalePrice'],axis=1)
y = train.LogSalePrice

In [26]:
xgb_model = xgb.XGBRegressor()

In [27]:
X,y = shuffle(X,y, random_state=42)
X = X.reset_index(drop=True)
y = y.reset_index(drop=True)

In [28]:
def rmse_cv(model,X,y):
    rmse = np.sqrt(-cross_val_score(model, X, y, scoring="neg_mean_squared_error", cv=6))
    return rmse

In [29]:
print(rmse_cv(xgb_model,X,y).mean())

0.13790232175471712


In [30]:
submit = test[['Id']]
submit = submit.reset_index(drop=True)

In [31]:
xgb_model.fit(X,y)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             n_estimators=100, n_jobs=None, num_parallel_tree=None,
             predictor=None, random_state=None, ...)

In [32]:
submit_predict = xgb_model.predict(test)
submit_predict = np.exp(submit_predict)

In [33]:
submit['SalePrice'] = submit_predict

In [35]:
submit.to_csv('submit_xgb_feature_eng.csv', index=False)

Reducing feature skewness actually slightly increased the Kaggle score. 